# Setup

In [ ]:
!pip install datasets
!pip install img2dataset
!wget https://huggingface.co/datasets/huggingface-projects/sd-multiplayer-data/resolve/main/rooms_data.db

# Make Dataset!

In [ ]:
%cd /content/
import sqlite3
import re
import itertools

f = open("path.txt", "w")
ff = open("prompt.txt", "w")
fff = open("time.txt","w")
# Create a SQL connection to our SQLite database
con = sqlite3.connect("/content/rooms_data.db")

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('SELECT key FROM rooms_data;'):
  s = "https://d26smi9133w0oo.cloudfront.net/" + str(row[0]) + "\n"
  f.write(s)

for row in cur.execute('SELECT time FROM rooms_data;'):  
  s = str(row[0])
  s = " |" + s[5:16] + "|\n"
  fff.write(s)

for row in cur.execute('SELECT prompt FROM rooms_data;'):  
  s = str(row[0])
  s = s.replace("jpg", "")
  s = s.replace("webp", "")
  words = s.split()
  s = (" ".join(words[1:])) + "\n" 
  ff.write(s)

f = open("path.txt", "r")
line = open("path.txt", "r")
ff = open("prompt.txt", "r")
fff = open("time.txt","r")
lines = line.readlines()

rfintext = open("realfinal.txt", "w")
rfintext.write("url,caption\n")

for Line in lines:
  path = f.readline()
  path = (path.replace("\n", ""))
  prompt = ff.readline()
  prompt = (prompt.replace("\n", ""))
  time = fff.readline()
  time = (time.replace("\n", ""))
  fin = path + ',"' + str(prompt) + str(time) + '"\n'
  rfintext.write(fin)

# Be sure to close the connection
con.close()

In [ ]:
# Review
%cd /content/
!head -n 1 prompt.txt
!head -n 3 realfinal.txt
!head -n 1 path.txt
!wc path.txt -l

In [ ]:
%cd /content/
!mkdir /content/imagefolder
%cd /content/imagefolder

In [ ]:
!img2dataset --url_list "/content/realfinal.txt" --input_format "csv" --url_col "url" --caption_col "caption" --thread_count=64 --image_size=256

In [ ]:
import glob
from datasets import *
from datasets import load_dataset
textdataset = sorted(glob.glob("/content/imagefolder/images/*****/*.txt"))
text = []
for i in textdataset:
  textfile = open(i, "r")
  textcontent = textfile.readline()
  textcontent = textcontent.replace('"', "")
  textcontent = textcontent.replace('\n', "")
  text.append(textcontent)
ims = sorted(glob.glob('/content/imagefolder/images/*****/*.jpg'))
d = {'image':ims, "text":text}
dataset = Dataset.from_dict(d).cast_column("image", Image()).cast_column("text",Value(dtype='string'))
print(dataset)

# Upload!

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
dataset.push_to_hub('-INSERTUSER-/OSD-Dataset')